# TPS Sampling Tutorial

In this tutorial, you will set up and run a transition path sampling simulation for the $C_{7eq}$ to $\alpha_R$ transition in alanine dipeptide. This is similar to the work in [Bolhuis, Dellago, and Chandler. PNAS **97**, 5877 (2000)](http://dx.doi.org/10.1073/pnas.100127697).

You will learn aspects of three different tools:

* [`OpenMM`](http://openmm.org), which you will use to perform the actual MD
* [`MDTraj`](http://mdtraj.org), which you will use to select atoms from the topology (similar to making Gromacs `idx` files) and to calculate collective variables
* [`OpenPathSampling`](http://openpathsampling.org), which will manage the path sampling simulation

You will also use a few other useful libraries:

* [`openmmtools`](https://github.com/choderalab/openmmtools), a library of additions that are not included in the OpenMM core
* [`matplotlib`](http://matplotlib.org/), a library for creating plots

We start off by importing the libraries we'll need. It's common to keep all the imports at the top of the file:

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from simtk.openmm import app
import simtk.openmm as mm
import simtk.unit as unit
import openmmtools

import openpathsampling as paths
import openpathsampling.engines.openmm as ops_openmm

import mdtraj as md

# Jupyter is raising deprecation warning from JSON now... ignore
import warnings
warnings.filterwarnings('ignore')

## Setting up the engine

Path sampling methods wrap around molecular dynamics, so one of the first things you need to do when setting up a path sampling simulation is to set up a molecular dynamics engine. In this example, we'll use OpenMM as the underlying engine.

The next cell shows you how to set up several things specific to OpenMM. We'll be running in the $NVT$ ensemble, with $T=300 K$. We're using the [Velocity Verlet with Velocity Randomization (VVVR) integrator](http://arxiv.org/abs/1301.3800), which simulates Langevin dynamics. Note that the integrator itself comes from [`openmmtools`](https://github.com/choderalab/openmmtools), a library that extends `OpenMM`.

You can learn a lot more about setting up OpenMM simulations from the [OpenMM documentation](http://docs.openmm.org/). However, it is often even easier to use the [OpenMM Script Builder](http://builder.openmm.org/) to learn how to set up the simulation the way you'd like.

In [2]:
# this cell is all OpenMM specific
forcefield = app.ForceField('amber96.xml', 'tip3p.xml')
system = forcefield.createSystem(
    topology=app.PDBFile("AD_initial_frame.pdb").topology, 
    nonbondedMethod=app.PME, 
    nonbondedCutoff=1.0*unit.nanometers,
    constraints=app.HBonds, 
    rigidWater=True,
    ewaldErrorTolerance=0.0005
)

integrator = openmmtools.integrators.VVVRIntegrator(
    300*unit.kelvin,
    1.0/unit.picoseconds,
    2.0*unit.femtoseconds
)
integrator.setConstraintTolerance(0.00001)

Next we deal with the interface between OpenMM and OPS. We create an OPS `Engine` object that wraps around the OpenMM system and integrator you just created.

In [3]:
openmm_properties = {}
engine_options = {
    'n_frames_max': 2000,
    'n_steps_per_frame': 10
}
ops_topology = ops_openmm.tools.topology_from_pdb("AD_initial_frame.pdb")
engine = ops_openmm.Engine(
    topology=ops_topology, 
    system=system, 
    integrator=integrator, 
    openmm_properties=openmm_properties,
    options=engine_options
)
engine.name = 'TPS MD Engine'

## Defining collective variables

We'll define our states based on the [Ramachandran angles](https://en.wikipedia.org/wiki/Ramachandran_plot) $\phi$ and $\psi$. These are dihedrals along the protein backbone, defined for residue $i$ as involving the following atoms:

* $\phi_i$: C$^{(i-1)}$, N$^{(i)}$, C$_\alpha^{(i)}$, C$^{(i)}$
* $\psi_i$: N$^{(i)}$, C$_\alpha^{(i)}$, C$^{(i)}$, N$^{(i+1)}$

where the parenthetical superscript indicates residue number. Note that the order of the atoms is important for the computation!

We'll need the atom numbers for these (just as Gromacs put those atom numbers into an `idx` file). To find them, we'll use MDTraj's `Topology` object. First, we convert the OPS `Topology` object to an MDTraj `Topology` object.

> NB: There are several types of `Topology` objects. See, for example, what you get from `type(engine.topology)` vs. `type(engine.simulation.topology)` and then `type(md_topology)` after you make it. The tricks discussed below are for the MDTraj topology objects.

In [ ]:
md_topology = md.Topology.from_openmm(engine.simulation.topology)

MDTraj has an advanced [atom selection language](http://mdtraj.org/1.8.0/atom_selection.html). Here we'll print out the backbone atoms and their atom numbers, since we know the atoms for our collective variables are from the backbone.

In [ ]:
for atom_number in md_topology.select("backbone"):
    print atom_number, md_topology.atom(atom_number)

There's only one pair of Ramachandran angles of interest here: the ones for residue 1, alanine. To figure out which atoms are needed, we use the definition of $\phi_i$, which tells us we need C$^{(0)}$, N$^{(1)}$, C$_\alpha^{(1)}$, and C$^{(1)}$; i.e., `ACE0-C`, `ALA1-N`, `ALA1-CA`, and `ALA1-C`. We can find the correct atom indices from the result of the previous cell. 

In [ ]:
# ACE0-C, ALA1-N, ALA1-CA, ALA1-C: 4,6,8,14
phi = paths.MDTrajFunctionCV("phi", md.compute_dihedrals, engine.topology, indices=[[4,6,8,14]])

In [ ]:
# YOUR TURN: Define `psi`, similarly to `phi` above.
# Figure out which atoms make the dihedral, find their atom indices, then create the CV.

## Defining the stable states

Once we've created our collective variables, we use them to define our stable states. For this system, we'll define the stable states as follows:

* $C_{7\text{eq}}$: $-180 < \phi < 0$ and $100 < \psi < 200$
* $\alpha_R$: $-180 < \phi < 0$ and $-100 < \psi < 0$

with the angles measured in degrees.

Note that OPS allows you to intersect two volumes with `&`.

In [ ]:
deg = 180.0/np.pi  # for conversion between radians and degrees

C_7eq = (
    paths.PeriodicCVDefinedVolume(phi, lambda_min=-180/deg, lambda_max=0/deg, 
                                  period_min=-np.pi, period_max=np.pi)
    & paths.PeriodicCVDefinedVolume(psi, lambda_min=100/deg, lambda_max=200/deg,
                                    period_min=-np.pi, period_max=np.pi)
).named("C_7eq")

In [ ]:
# YOUR TURN: define the `alpha_R` state

You may have noticed that, even though the angles are defined from -180 degrees to 180 degrees, the C$_{7\text{eq}}$ state is defined $\psi$ up to 200 degrees. `PeriodicCVDefinedVolume` knows how to correctly handle volumes that wrap around a periodic boundary.

## Setting up the reaction network and move scheme

In OPS, path sampling simulations sample networks of reactions (even if the network only involves a simple $A\to B$ transition, as is the case here). For (flexible length) transition path sampling, use the `TPSNetwork` object.

OPS also allows you to set up many different kinds of move schemes. In flexible length transition path sampling, there's usually only one mover in the move scheme: a shooting mover. The `OneWayShootingMoveScheme` sets that up quickly. We will use uniform shooting point selection.

In [ ]:
network = paths.TPSNetwork(initial_states=C_7eq, final_states=alpha_R)
scheme = paths.OneWayShootingMoveScheme(network=network, 
                                        selector=paths.UniformSelector(),
                                        engine=engine)

## Getting an initial trajectory

In practice, getting the initial trajectory can be one of the hardest parts of path sampling. For this example, you could easily do it by running high temperature MD to get an unphysical path, and then equilibrate it for a while. However, for the purposes of this tutorial, I already did that for you. So we'll just load the equilibrated trajectory from a file.

In [ ]:
init_traj_storage = paths.Storage("initial_trajectory.nc", 'r')
init_traj = init_traj_storage.trajectories[0]

In [ ]:
plt.plot(phi(init_traj), psi(init_traj))
plt.xlabel("$\phi$")
plt.ylabel("$\psi$");

The next step associates the initial trajectory with the ensemble we'll be sampling. This becomes important when there are multiple ensembles, as with transition interface sampling, and you must ensure that the initial trajectories are correct for each ensemble.

In [ ]:
initial_conditions = scheme.initial_conditions_from_trajectories(init_traj)

## Running the simulation

Now all the parts in place, and we can run the simulation! First, we open a file to store the results in, and then we build the simulation and run it.

In [ ]:
storage = paths.Storage("tps_simulation_results.nc", "w", template=init_traj[0])
sampler = paths.PathSampling(storage=storage,
                             move_scheme=scheme,
                             sample_set=initial_conditions)

In [ ]:
sampler.run(40)

In [ ]:
storage.close()